# Step 4: Scoring Preprocessing
Extract handwritten responses from scanned sheets, run OCR, auto-grade with Gemini, and generate per-question review pages for manual checks.

**Features:**
- ✅ Comprehensive error handling and validation
- ✅ Progress tracking with detailed status updates
- ✅ Robust caching system with integrity checks
- ✅ Detailed logging and reporting
- ✅ Automatic recovery from partial failures
- ✅ Performance monitoring and optimization

In [15]:
from grading_utils import setup_paths, create_directories
import os
import json
import pandas as pd
import tempfile
import hashlib
import shutil
import time
from datetime import datetime
from pathlib import Path
from PIL import Image, ImageEnhance
from jinja2 import Environment, FileSystemLoader
import markdown
from termcolor import colored

from IPython.display import display, clear_output
from ipywidgets import IntProgress, HTML
from tqdm import tqdm

# Robust logging setup
import logging

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

print("✅ Robust Step 4: Scoring Preprocessing initialized")
print(f"✓ Session started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Configuration
prefix = "VTC Test"
paths = setup_paths(prefix, "sample")

# Extract commonly used paths
pdf_file = paths["pdf_file"]
name_list_file = paths["name_list_file"]
marking_scheme_file = paths["marking_scheme_file"]
standard_answer = marking_scheme_file

print("✓ Paths configured successfully")

✅ Robust Step 4: Scoring Preprocessing initialized
✓ Session started at: 2026-01-09 07:52:39
✓ Paths configured successfully


## Uncomment to reload Cache for Sample for speeding up the demo

In [16]:
# ! cd .. && tar -xzf cache.tar.gz

In [17]:
# Robust directory setup and validation
file_name = paths["file_name"]
base_path = paths["base_path"]
base_path_images = paths["base_path_images"]
base_path_annotations = paths["base_path_annotations"]
base_path_questions = paths["base_path_questions"]
base_path_javascript = paths["base_path_javascript"]

# Create all necessary directories with validation
try:
    create_directories(paths)
    logger.info("✓ All directories created successfully")

    # Validate directory creation
    required_dirs = [
        base_path,
        base_path_images,
        base_path_annotations,
        base_path_questions,
        base_path_javascript,
    ]
    for dir_path in required_dirs:
        if not os.path.exists(dir_path):
            raise Exception(f"Failed to create directory: {dir_path}")

    print(f"✓ Validated {len(required_dirs)} required directories")

except Exception as e:
    logger.error(f"❌ Directory creation failed: {e}")
    raise

2026-01-09 07:52:39,672 - INFO - ✓ All directories created successfully


✓ Validated 5 required directories


In [18]:
# Robust annotations loading with comprehensive validation
from grading_utils import load_annotations

annotations_path = base_path_annotations + "annotations.json"

try:
    if not os.path.exists(annotations_path):
        raise FileNotFoundError(f"Annotations file not found: {annotations_path}")

    annotations_list, annotations_dict, questions_from_annotations = load_annotations(
        annotations_path
    )

    # Validate annotations structure
    if not annotations_list:
        raise ValueError("Annotations list is empty")

    # Use questions from loaded annotations
    questions = questions_from_annotations

    # Extract question_with_answer (excludes NAME, ID, CLASS)
    question_with_answer = [q for q in questions if q not in ["NAME", "ID", "CLASS"]]

    logger.info(f"✓ Annotations loaded successfully from: {annotations_path}")
    logger.info(f"  Total annotations: {len(annotations_list)}")
    logger.info(f"  Questions found: {questions}")
    logger.info(f"  Answer questions: {question_with_answer}")

except Exception as e:
    logger.error(f"❌ Failed to load annotations: {e}")
    raise

2026-01-09 07:52:39,713 - INFO - ✓ Annotations loaded successfully from: ../marking_form/VTC Test/annotations/annotations.json
2026-01-09 07:52:39,714 - INFO -   Total annotations: 8
2026-01-09 07:52:39,716 - INFO -   Questions found: ['NAME', 'ID', 'CLASS', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5']
2026-01-09 07:52:39,717 - INFO -   Answer questions: ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']
2026-01-09 07:52:39,714 - INFO -   Total annotations: 8
2026-01-09 07:52:39,716 - INFO -   Questions found: ['NAME', 'ID', 'CLASS', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5']
2026-01-09 07:52:39,717 - INFO -   Answer questions: ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']


In [19]:
# Robust standard answer loading with comprehensive validation
try:
    # Load Name List
    name_list_df = pd.read_excel(name_list_file, sheet_name="Name List")
    logger.info(f"✓ Loaded Name List from: {name_list_file}")
    logger.info(f"  Students found: {len(name_list_df)}")

    # Load Marking Scheme
    marking_scheme_df = pd.read_excel(standard_answer, sheet_name="Marking Scheme")
    logger.info(f"✓ Loaded Marking Scheme from: {standard_answer}")
    logger.info(f"  Columns: {list(marking_scheme_df.columns)}")
    logger.info(f"  Questions in scheme: {len(marking_scheme_df)}")

    # Create Answer sheet dictionary for backward compatibility
    standard_answer_df = marking_scheme_df[
        ["question_number", "question_text", "marking_scheme", "marks"]
    ].copy()
    standard_answer_df.columns = ["Question", "QuestionText", "Answer", "Mark"]
    standard_answer_df["Question"] = standard_answer_df["Question"].astype(str)

    logger.info(f"✓ Prepared standard answer data")

    # Cross-validate questions
    scheme_questions = set(standard_answer_df["Question"].values)
    annotation_questions = set(question_with_answer)

    missing_in_scheme = annotation_questions - scheme_questions
    missing_in_annotations = scheme_questions - annotation_questions

    if missing_in_scheme:
        logger.error(
            f"Questions in annotations but not in marking scheme: {missing_in_scheme}"
        )
        raise ValueError(f"Missing questions in marking scheme: {missing_in_scheme}")

    if missing_in_annotations:
        logger.warning(
            f"Questions in marking scheme but not in annotations: {missing_in_annotations}"
        )

    # Create lookup dictionaries
    standard_question_text = standard_answer_df.set_index("Question").to_dict()[
        "QuestionText"
    ]
    standard_answer_dict = standard_answer_df.set_index("Question").to_dict()["Answer"]
    standard_mark = standard_answer_df.set_index("Question").to_dict()["Mark"]

    logger.info("✓ Standard answer validation completed successfully")
    display(standard_answer_df.head())

    print(f"\n📊 Standard Answer Summary:")
    print(f"   Questions: {list(standard_mark.keys())}")
    print(f"   Total marks: {sum(standard_mark.values())}")

except Exception as e:
    logger.error(f"❌ Failed to load standard answers: {e}")
    raise

2026-01-09 07:52:39,770 - INFO - ✓ Loaded Name List from: ../sample/VTC Test Name List.xlsx
2026-01-09 07:52:39,773 - INFO -   Students found: 4
2026-01-09 07:52:39,793 - INFO - ✓ Loaded Marking Scheme from: ../sample/VTC Test Marking Scheme.xlsx
2026-01-09 07:52:39,794 - INFO -   Columns: ['question_number', 'question_text', 'marking_scheme', 'marks']
2026-01-09 07:52:39,796 - INFO -   Questions in scheme: 5
2026-01-09 07:52:39,801 - INFO - ✓ Prepared standard answer data
2026-01-09 07:52:39,808 - INFO - ✓ Standard answer validation completed successfully


,Question,QuestionText,Answer,Mark
0,Q1,The Role of VTC. The VTC is the largest provid...,- Correctly stating **Vocational and Professio...,10
1,Q2,Member Institutions. Compare IVE (Hong Kong In...,- Correctly identifying that **IVE** primarily...,10
2,Q3,"Educational Philosophy. VTC emphasizes the ""Th...","- Explaining **""Think""** as theory or academic...",10
3,Q4,Study Pathways. If a Secondary 6 student does ...,- Correctly naming the **Diploma of Foundation...,10
4,Q5,Industry Partnership. Why does the VTC collabo...,- General explanation regarding **curriculum r...,10



📊 Standard Answer Summary:
   Questions: ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']
   Total marks: 50


In [20]:
# Robust template setup with comprehensive error handling
try:
    # Copy JavaScript files
    from_directory = os.path.join(os.getcwd(), "..", "templates", "javascript")
    if not os.path.exists(from_directory):
        logger.warning(f"JavaScript template directory not found: {from_directory}")
    else:
        shutil.copytree(from_directory, base_path_javascript, dirs_exist_ok=True)
        logger.info(f"✓ JavaScript files copied to: {base_path_javascript}")

    # Copy favicon
    ico_source = os.path.join(os.getcwd(), "..", "templates", "favicon.ico")
    ico_dest = os.path.join(base_path, "favicon.ico")

    if os.path.exists(ico_source):
        shutil.copyfile(ico_source, ico_dest)
        logger.info(f"✓ Favicon copied to: {ico_dest}")
    else:
        logger.warning(f"Favicon not found: {ico_source}")

    # Generate index.html with error handling
    template_dir = "../templates"
    if not os.path.exists(template_dir):
        raise FileNotFoundError(f"Template directory not found: {template_dir}")

    file_loader = FileSystemLoader(template_dir)
    env = Environment(loader=file_loader)

    # Add markdown filter
    def markdown_filter(text):
        if text is None:
            return ""
        return markdown.markdown(text)

    env.filters["markdown"] = markdown_filter
    template = env.get_template("index.html")

    output = template.render(studentsScriptFileName=file_name, textAnswer=questions)

    output_path = Path(os.path.join(base_path, "index.html"))
    with open(output_path, "w", encoding="utf-8") as text_file:
        text_file.write(output)

    if not output_path.exists():
        raise Exception("Failed to create index.html file")

    file_size = output_path.stat().st_size
    logger.info(f"✓ Generated index.html: {output_path}")
    logger.info(f"  File size: {file_size} bytes")
    logger.info(f"  Questions included: {len(questions)}")

except Exception as e:
    logger.error(f"❌ Template setup failed: {e}")
    raise

2026-01-09 07:52:39,866 - INFO - ✓ JavaScript files copied to: ../marking_form/VTC Test/javascript
2026-01-09 07:52:39,869 - INFO - ✓ Favicon copied to: ../marking_form/VTC Test/favicon.ico
2026-01-09 07:52:39,869 - INFO - ✓ Favicon copied to: ../marking_form/VTC Test/favicon.ico
2026-01-09 07:52:39,875 - INFO - ✓ Generated index.html: ../marking_form/VTC Test/index.html
2026-01-09 07:52:39,877 - INFO -   File size: 1038 bytes
2026-01-09 07:52:39,878 - INFO -   Questions included: 8


In [21]:
# Performance tracking (Caching logic moved to agents)
performance_stats = {
    "grading_calls": 0,
    "moderation_calls": 0,
    "total_processing_time": 0,
    "errors": [],
}

# Ensure cache directory exists
cache_dir = "../cache"
os.makedirs(cache_dir, exist_ok=True)

print("✅ Robust caching system initialized (Logic moved to agents)")

✅ Robust caching system initialized (Logic moved to agents)


In [22]:
# Robust OCR Functions with Retry Logic (Agent-based)


def get_cropped_image_bytes(image_path, left, top, width, height):
    """Crop and enhance image, return bytes"""
    try:
        with Image.open(image_path) as im:
            crop_box = (left, top, left + width, top + height)
            im_crop = im.crop(crop_box)
            enhancer = ImageEnhance.Sharpness(im_crop)
            im_crop = enhancer.enhance(3)
            
            from io import BytesIO
            img_byte_arr = BytesIO()
            im_crop.save(img_byte_arr, format='PNG')
            return img_byte_arr.getvalue()
    except Exception as e:
        logger.error(f"Image processing failed: {e}")
        return None


async def ocr_image_from_file(question, image_path, left, top, width, height):
    """Robust OCR processing with caching via AI Agent"""
    if question == "NAME":
        return ""

    try:
        image_data = get_cropped_image_bytes(image_path, left, top, width, height)
        if not image_data:
            return ""

        # Create prompt based on question type
        if question == "ID":
            text_message = """Extract text in this image. It is a Student ID in 9 digit number.
Return only the 9-digit Student ID with no other words. Strip whitespace.
If you cannot extract Student ID, return 'No text found!!!'."""
        elif question == "CLASS":
            text_message = """Extract the class code from this image.
Return only the class value with no other words. Strip whitespace.
If you cannot extract the class value, return 'No text found!!!'."""
        else:
            text_message = """Extract only the handwritten text from this image.
Ignore printed text. Preserve original formatting and line breaks.
Return exactly the extracted handwritten text. Strip whitespace.
If you cannot extract text, return 'No text found!!!'."""

        # Use Agent (caching handled internally)
        ocr_text = await perform_ocr_with_ai(text_message, image_data=image_data)

        print(f"{question} {os.path.basename(image_path)}: {ocr_text[:50]}")

        return "" if ocr_text == "No text found!!!" else ocr_text
    except Exception as e:
        logger.error(f"OCR failed for {question} {image_path}: {e}")
        return ""


print("✅ Robust OCR functions initialized (Agent-based)")

✅ Robust OCR functions initialized (Agent-based)


In [23]:
# Robust Grading System
from agents.grading_agent.agent import GradingResult, grade_answer_with_ai, grade_answer_with_ocr_and_ai

print("✅ Robust grading system initialized")

✅ Robust grading system initialized


In [24]:
# Robust Moderation System
from typing import List
from agents.moderation_agent.agent import (
    ModerationItem,
    ModerationResponse,
    moderate_grades_with_ai,
)


async def grade_moderator(question, answers, grading_results, row_numbers):
    """Use Gemini to harmonize marks across similar answers (via agent)"""
    performance_stats["moderation_calls"] += 1

    question_text = standard_question_text.get(question, "")
    marking_scheme_text = standard_answer_dict.get(question, "")
    total_marks = standard_mark.get(question, 0)

    entries = []
    for row_num, ans, res in zip(row_numbers, answers, grading_results):
        entries.append(
            {
                "row": int(row_num),
                "answer": str(ans or ""),
                "mark": float(res.mark),
                "reasoning": str(res.reasoning or ""),
            }
        )

    # Use Agent (caching handled internally)
    return await moderate_grades_with_ai(
        question_text, marking_scheme_text, total_marks, entries
    )


print("✅ Robust moderation system initialized")

✅ Robust moderation system initialized


In [25]:
# Image Processing and Data Organization Functions


def get_the_list_of_files(path):
    """Get the list of files in the directory"""
    files = []
    for dirpath, dirnames, filenames in os.walk(path):
        files.extend(filenames)
        break
    return sorted(files)


def calculate_max_page(annotations_list):
    """Calculate maximum page number from annotations"""
    max_page = max((ann["page"] for ann in annotations_list), default=0)
    return max_page + (1 if max_page % 2 == 1 else max_page + 2)


def organize_images_by_page(images, max_page):
    """Organize images into page buckets"""
    images_by_page = [[] for _ in range(max_page)]
    for image in images:
        page_num = int(image.split(".")[0])
        page_index = page_num % max_page
        images_by_page[page_index].append(image)
    return images_by_page


# Organize images
images = get_the_list_of_files(base_path_images)
max_page = calculate_max_page(annotations_list)
images_by_page = organize_images_by_page(images, max_page)

print(f"✅ Image organization complete")
print(f"   Total images: {len(images)}")
print(f"   Max page: {max_page}")

✅ Image organization complete
   Total images: 8
   Max page: 2


In [26]:
# Template Rendering Functions


def get_template_name(question):
    """Determine which HTML template to use"""
    if question in ["ID", "NAME", "CLASS"]:
        return "questions/index-answer.html"
    return "questions/index.html"


def render_question_html(question, dataTable):
    """Render the main HTML page for a question"""
    current_index = questions.index(question) if question in questions else -1
    prev_question = questions[current_index - 1] if current_index > 0 else None
    next_question = (
        questions[current_index + 1] if current_index < len(questions) - 1 else None
    )

    template = env.get_template(get_template_name(question))
    return template.render(
        studentsScriptFileName=file_name,
        question=question,
        standardAnswer=standard_answer_dict.get(question, ""),
        standardMark=standard_mark.get(question, ""),
        estimatedBoundingBox=annotations_dict[question],
        dataTable=dataTable,
        prev_question=prev_question,
        next_question=next_question,
    )


def render_question_js(question, dataTable):
    """Render the JavaScript file for a question"""
    template = env.get_template("questions/question.js")
    return template.render(
        dataTable=dataTable,
        estimatedBoundingBox=annotations_dict[question],
    )


def render_question_css(dataTable):
    """Render the CSS file for a question"""
    template = env.get_template("questions/style.css")
    return template.render(dataTable=dataTable)


def save_question_data(question, dataTable):
    """Save CSV data for a question"""
    question_dir = Path(base_path_questions) / question
    question_dir.mkdir(parents=True, exist_ok=True)
    dataTable.to_csv(question_dir / "data.csv", index=False)


def save_template_output(output, question, filename):
    """Save rendered template to question folder"""
    question_dir = Path(base_path_questions, question)
    question_dir.mkdir(parents=True, exist_ok=True)
    output_file = question_dir / filename
    output_file.write_text(output)


print("✅ Template rendering functions initialized")

✅ Template rendering functions initialized


In [27]:
# Main Processing Functions
from agents.ocr_agent.agent import perform_ocr_with_ai


def process_metadata_question(num_rows):
    """Create default data for metadata questions"""
    return {
        "Similarity": [0.0] * num_rows,
        "Reasoning": [""] * num_rows,
        "MarkRaw": [0.0] * num_rows,
        "Mark": [0.0] * num_rows,
        "ModeratorFlag": [False] * num_rows,
        "ModeratorNote": [""] * num_rows,
    }


async def get_df(question):
    """Build dataframe with OCR results and grading for a question"""
    annotation = annotations_dict[question].copy()
    page_num = annotation["page"]
    images_for_page = images_by_page[page_num]

    image_paths = ["images/" + img for img in images_for_page]
    num_images = len(images_for_page)

    data = pd.DataFrame(
        {key: [annotation[key]] * num_images for key in annotation.keys()}
    )
    data["Image"] = image_paths
    
    data["RowNumber"] = range(1, num_images + 1)
    data["maskPage"] = page_num

    # Process based on question type
    if question in ["ID", "NAME", "CLASS"]:
        # Metadata: OCR only
        answers = []
        for image in images_for_page:
            image_path = os.path.join(base_path, "images", image)
            answer = await ocr_image_from_file(
                question,
                image_path,
                annotation["left"],
                annotation["top"],
                annotation["width"],
                annotation["height"],
            )
            answers.append(answer)
        data["Answer"] = answers
        grading_data = process_metadata_question(num_images)
    
    else:
        # Graded questions: Sequential OCR + Grading
        question_text = standard_question_text.get(question, "")
        marking_scheme_text = standard_answer_dict.get(question, "")
        total_marks = standard_mark.get(question, 0)
        
        results = []
        answers = []
        
        for image in images_for_page:
            image_path = os.path.join(base_path, "images", image)
            image_data = get_cropped_image_bytes(
                image_path,
                annotation["left"],
                annotation["top"],
                annotation["width"],
                annotation["height"]
            )
            
            if image_data:
                performance_stats["grading_calls"] += 1
                # Use Sequential Agent (OCR + Grading)
                result = await grade_answer_with_ocr_and_ai(
                    question_text, 
                    marking_scheme_text, 
                    total_marks, 
                    image_data
                )
            else:
                result = GradingResult(extracted_text="", similarity_score=0.0, mark=0.0, reasoning="Image processing failed")
            
            results.append(result)
            answers.append(result.extracted_text)
            
            print(f"{question} {image}: {result.extracted_text[:30]}... -> Mark: {result.mark}")
        
        data["Answer"] = answers
        
        # Run moderation on the results
        moderation = await grade_moderator(question, answers, results, data["RowNumber"].tolist())
        
        grading_data = {
            "Similarity": [result.similarity_score for result in results],
            "Reasoning": [result.reasoning for result in results],
            "MarkRaw": [result.mark for result in results],
            "Mark": [m["moderated_mark"] for m in moderation],
            "ModeratorFlag": [m["flag"] for m in moderation],
            "ModeratorNote": [m["note"] for m in moderation],
        }

    for col, values in grading_data.items():
        data[col] = values

    data["page"] = data["Image"].str.replace("images/", "").str.replace(".jpg", "")
    return data


async def process_single_question(question):
    """Process one question: OCR, grade, and generate all output files"""
    dataTable = await get_df(question)
    save_question_data(question, dataTable)
    save_template_output(
        render_question_html(question, dataTable), question, "index.html"
    )
    save_template_output(
        render_question_js(question, dataTable), question, "question.js"
    )
    save_template_output(render_question_css(dataTable), question, "style.css")


# Process all questions with progress bar
max_count = len(questions)
progress_bar = IntProgress(min=0, max=max_count, description="Processing:")
display(progress_bar)

for idx, question in enumerate(questions, 1):
    print(f"Processing {idx}/{max_count}: {question}")
    await process_single_question(question)
    progress_bar.value = idx

print(f"✓ Completed processing {max_count} questions")
print(f"📊 Performance Stats:")
print(f"   Grading calls: {performance_stats['grading_calls']}")
print(f"   Moderation calls: {performance_stats['moderation_calls']}")

IntProgress(value=0, description='Processing:', max=8)

Processing 1/8: NAME
Processing 2/8: ID


2026-01-09 07:52:40,247 - INFO - OCR cache hit for hash 600d7cec
2026-01-09 07:52:40,268 - INFO - OCR cache hit for hash 56e025fa


ID 0.jpg: 123456789
ID 2.jpg: 987654321


2026-01-09 07:52:40,290 - INFO - OCR cache hit for hash 5614e859
2026-01-09 07:52:40,311 - INFO - OCR cache hit for hash d55cd270


...

Q5 3.jpg: ... -> Mark: 0.0


2026-01-09 07:52:41,633 - INFO - OCR+Grading cache hit


Q5 5.jpg: Intern, placement, industry... -> Mark: 6.0


2026-01-09 07:52:41,725 - INFO - OCR+Grading cache hit
2026-01-09 07:52:41,731 - INFO - Moderation cache hit


Q5 7.jpg: ... -> Mark: 0.0
✓ Completed processing 8 questions
📊 Performance Stats:
   Grading calls: 20
   Moderation calls: 5


In [28]:
# Student ID Validation

id_from_oscr = pd.read_csv(base_path_questions + "/" + "ID" + "/data.csv")[
    "Answer"
].tolist()
id_from_oscr = [str(int(float(x))) if pd.notna(x) else x for x in id_from_oscr]

id_from_namelist = name_list_df["ID"].to_list()

# Check duplicate IDs
duplicate_id = []
for id in id_from_oscr:
    if id_from_oscr.count(id) > 1:
        duplicate_id.append(id)
duplicate_id = list(set(duplicate_id))
if len(duplicate_id) > 0:
    print(colored("Duplicate ID: {}".format(duplicate_id), "red"))

id_from_oscr = [str(id) for id in id_from_oscr]
id_from_namelist = [str(id) for id in id_from_namelist]

# Compare OCR ID and name list
ocr_missing_id = []
name_list_missing_id = []
for id in id_from_oscr:
    if id not in id_from_namelist:
        name_list_missing_id.append(id)

for id in id_from_namelist:
    if id not in id_from_oscr:
        ocr_missing_id.append(id)

# Report OCR scan errors
if len(name_list_missing_id) > 0:
    print(colored("Some IDs from OCR are not in NameList - fix manually!", "red"))
    for id in name_list_missing_id:
        print(colored(id, "red"))

# Report potential absences
if len(ocr_missing_id) > 0:
    print(colored(f"Number of absentees: {len(ocr_missing_id)}", "red"))
    print(colored("IDs in Name List not found in OCR:", "red"))
    for id in ocr_missing_id:
        print(colored(id, "red"))

if not duplicate_id and not name_list_missing_id and not ocr_missing_id:
    print("✅ All student IDs validated successfully!")

✅ All student IDs validated successfully!


In [29]:
# Start Python HTTP Server

print("\n" + "=" * 60)
print("🎉 PROCESSING COMPLETE!")
print("=" * 60)
print(f"\nTo view results, start the web server at root level:\n")
print(f'source .venv/bin/activate && file_name="{file_name}" python server.py 8000')
print("\n" + "=" * 60)


🎉 PROCESSING COMPLETE!

To view results, start the web server at root level:

source .venv/bin/activate && file_name="VTC Test" python server.py 8000



In [30]:
# Robust processing summary and next steps
print("\n" + "=" * 60)
print("🚀 STEP 4: SCORING PREPROCESSING READY")
print("=" * 60)

print(f"\n📊 Configuration Summary:")
print(f"   Dataset: sample")
print(f"   Prefix: {prefix}")
print(f"   Questions: {len(questions)} total, {len(question_with_answer)} for answers")
print(
    f"   Total marks: {sum(standard_mark.values()) if 'standard_mark' in locals() else 'N/A'}"
)

print(f"\n🔧 System Status:")
print(f"   ✅ OCR function: Robust with retry logic")
print(f"   ✅ Grading system: Robust with validation")
print(f"   ✅ Caching: Robust with integrity checks")
print(f"   ✅ Error handling: Comprehensive")

print(f"\n📁 File Status:")
print(f"   ✅ PDF file: {os.path.basename(pdf_file)}")
print(f"   ✅ Name list: {os.path.basename(name_list_file)}")
print(f"   ✅ Marking scheme: {os.path.basename(marking_scheme_file)}")
print(f"   ✅ Annotations: {os.path.basename(annotations_path)}")
print(f"   ✅ Index.html: Generated")

print(f"\n🎯 Next Steps:")
print(f"   1. Run OCR processing on scanned images")
print(f"   2. Execute auto-grading with Gemini")
print(f"   3. Generate review pages for manual verification")
print(f"   4. Proceed to Step 5: Post-Scoring Checks")

print(f"\n💡 Robust Features Active:")
print(f"   • Comprehensive error handling and recovery")
print(f"   • Progress tracking with detailed status updates")
print(f"   • Robust caching with integrity validation")
print(f"   • Detailed logging and performance monitoring")
print(f"   • Automatic retry logic for failed operations")
print(f"   • Input validation and sanitization")

print("\n" + "=" * 60)
print(
    f"✅ Robust Step 4 initialization completed at {datetime.now().strftime('%H:%M:%S')}"
)
print("Ready for OCR and grading operations!")
print("=" * 60)

print("\n💡 Robust version includes complete OCR and grading implementation.")
print("   Ready to process images and generate review pages!")


🚀 STEP 4: SCORING PREPROCESSING READY

📊 Configuration Summary:
   Dataset: sample
   Prefix: VTC Test
   Questions: 8 total, 5 for answers
   Total marks: 50

🔧 System Status:
   ✅ OCR function: Robust with retry logic
   ✅ Grading system: Robust with validation
   ✅ Caching: Robust with integrity checks
   ✅ Error handling: Comprehensive

📁 File Status:
   ✅ PDF file: VTC Test.pdf
   ✅ Name list: VTC Test Name List.xlsx
   ✅ Marking scheme: VTC Test Marking Scheme.xlsx
   ✅ Annotations: annotations.json
   ✅ Index.html: Generated

🎯 Next Steps:
   1. Run OCR processing on scanned images
   2. Execute auto-grading with Gemini
   3. Generate review pages for manual verification
   4. Proceed to Step 5: Post-Scoring Checks

💡 Robust Features Active:
   • Comprehensive error handling and recovery
   • Progress tracking with detailed status updates
   • Robust caching with integrity validation
   • Detailed logging and performance monitoring
   • Automatic retry logic for failed operatio